In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
with open("stop_words_en.txt") as f1:
    x = f1.readlines()
def strip(x):
    return x.rstrip('\r\n')        
stop_list_2 = map(strip,x)    


for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word in stop_list_2:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

#print >> sys.stderr, "reporter:counter:group,Total words/Total stop words"


Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys


current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
%%writefile reducer_2.py

import sys

current_key = None
word_sum = 0
total_sum = 0
total_stop = 0

Overwriting reducer_2.py


In [5]:
%%writefile -a reducer_2.py

with open("stop_words_en.txt") as f1:
    x = f1.readlines()
def strip(x):
    return x.rstrip('\r\n')        
stop_list_2 = map(strip,x)    

stop_list = [1]
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    total_sum+= count
    if key in stop_list_2:
        total_stop+=count
#print "%s\t%d" % (total_sum, total_stop)
print total_stop* 100./total_sum

Appending to reducer_2.py


In [6]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [7]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,reducer_2.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer_2.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

41.6028241522	


rm: `wordcount_result_1513931958457216': No such file or directory
17/12/22 08:39:21 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/22 08:39:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/22 08:39:24 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/22 08:39:24 INFO mapreduce.JobSubmitter: number of splits:2
17/12/22 08:39:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1513917445282_0015
17/12/22 08:39:24 INFO impl.YarnClientImpl: Submitted application application_1513917445282_0015
17/12/22 08:39:24 INFO mapreduce.Job: The url to track the job: http://371a33cc52da:8088/proxy/application_1513917445282_0015/
17/12/22 08:39:24 INFO mapreduce.Job: Running job: job_1513917445282_0015
17/12/22 08:39:31 INFO mapreduce.Job: Job job_1513917445282_0015 running in uber mode : false
17/12/22 08:39:31 INFO mapreduce.Job:  map 0% reduce 0%
17/12/22 08:39:47 INFO mapreduce.Job:  map 4% reduce 0%
17/12/22 08:39